In [1]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.animation as animation
from sklearn.metrics import r2_score

from scipy.integrate import solve_ivp

%matplotlib inline

## Generating data

In [2]:
L = 4
M = 2
G = 9.8
simulation_duration = 6

dt = .001
rk_integrator_args = {}
rk_integrator_args['rtol'] = 1e-13
rk_integrator_args['method'] = 'RK45'
rk_integrator_args['atol'] = 1e-10

odeint_integrator_args = {}
odeint_integrator_args['rtol'] = 1e-13
odeint_integrator_args['atol'] = 1e-10
odeint_integrator_args['full_output'] = 1

In [3]:
def pendulum_motion(t, input):
    theta, omega = input
    d_theta = omega
    d_omega = (-G/L) * np.sin(theta)
    
    d_f = np.array([d_theta, d_omega])
        
    return d_f

In [4]:
def compute_joints_position(thetas: np.ndarray):
    x_1,y_1 = np.array([[L * np.sin(theta), -L * np.cos(theta)] for theta in thetas]).T
    
    return np.array([x_1, y_1])

In [5]:
#returns (thetas, coordinates_over_time, thetas+omegas)
def compute_thetas_over_time(duration, dt, initial_state) -> tuple:
    time = np.arange(0, duration,dt)
    t_span = (time[0], time[-1])
    x_train = solve_ivp(pendulum_motion, t_span, initial_state, t_eval=time, **rk_integrator_args).y
    
    x_train_theta = x_train[0]
    joints_over_time = compute_joints_position(x_train_theta)
    
    return x_train_theta,joints_over_time,x_train

In [6]:
t_train = np.arange(0, simulation_duration, dt)
initial_pendulum_config = np.array([np.deg2rad(80), 0])
x_train_theta ,train_joints_over_time,x_train = compute_thetas_over_time(simulation_duration, dt, initial_pendulum_config)


In [7]:
t_test = np.arange(0, simulation_duration, dt)
test_initial_pendulum_config = np.array([np.deg2rad(65),0])
x_test_theta ,test_joints_over_time,x_test = compute_thetas_over_time(simulation_duration, dt, test_initial_pendulum_config)

In [8]:
true_derivatives = []
for i in range(len(t_train)):
    true_derivatives.append(pendulum_motion(t_train[i], x_train.T[i]))
    
true_derivatives=np.array(true_derivatives)
print(true_derivatives.shape)

(6000, 2)


In [9]:
def animate_pendulum_versus(data1, data2, filename='single_pendulum_versus.mp4', interval=50, fps=20):
    x1, y1 = data1
    x2, y2 = data2

    n_frames = len(x1)
    
    fig, ax = plt.subplots()
    all_x = [x1, x2]
    all_y = [y1, y2]
    ax.set_xlim(min(map(np.min, all_x)) - 1, max(map(np.max, all_x)) + 1)
    ax.set_ylim(min(map(np.min, all_y)) - 1, max(map(np.max, all_y)) + 1)
    ax.set_aspect('equal')
    ax.set_title('Sinlge Pendulums Side by Side')

    line1, = ax.plot([], [], 'x-', lw=2, label="True pendulum", color='blue')
    line2, = ax.plot([], [], 'o-', lw=2, label="Estimated pendulum", color='red')
    ax.legend()

    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        return line1, line2

    def update(i):
        # Pendulum 1
        x_vals1 = [0, x1[i]]
        y_vals1 = [0, y1[i]]
        line1.set_data(x_vals1, y_vals1)

        # Pendulum 2
        x_vals2 = [0, x2[i]]
        y_vals2 = [0, y2[i]]
        line2.set_data(x_vals2, y_vals2)

        return line1, line2

    ani = animation.FuncAnimation(fig, update, frames=n_frames,
                                  init_func=init, blit=True, interval=interval)

    writer = animation.FFMpegWriter(fps=fps)
    ani.save(filename, writer=writer)
    plt.close(fig)


In [10]:
def draw_state_diagrams(true_thetas, estimated_thetas, time_points, plot_file_name):
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(15, 7))
    
    ax.plot(time_points, true_thetas, '-.', lw=2, label="True pendulum", color='blue')
    ax.plot(time_points, estimated_thetas, '--', lw=1, label="Estimated pendulum", color='red')
    plt.title("Pendulum angle over time")
    plt.xlabel("Time")
    plt.ylabel("Theta")
    plt.legend()
    
    plt.savefig(plot_file_name, dpi=1300)
    plt.close()
    

In [11]:
def loop_around_angles(theta: np.ndarray):
    return (np.abs(theta) % (2 * np.pi)) * np.sign(theta)

In [12]:
def animate_pendulum_single(data, filename='single_pendulum.mp4', interval=50, fps=20):
    x1, y1 = data
    n_frames = data.shape[1]

    fig, ax = plt.subplots()
    ax.set_xlim(np.min(x1) - 2, np.max(x1) + 2)
    ax.set_ylim(np.min(y1) - 2, np.max(y1) + 2)   
    ax.set_aspect('equal')
    ax.set_title('Single Pendulum Animation')

    line, = ax.plot([], [], 'o-', lw=2)

    def init():
        line.set_data([], [])
        return line,

    def update(i):
        this_x = [0, x1[i]]
        this_y = [0, y1[i]]
        line.set_data(this_x, this_y)
        return line,

    ani = animation.FuncAnimation(fig, update, frames=n_frames,
                                  init_func=init, blit=True, interval=interval)

    writer = animation.FFMpegWriter(fps=fps)
    ani.save(filename, writer=writer)
    plt.close(fig)


In [13]:
def plot_progressive_erros(real_theta, estimated_theta, time_points, file_name="Error"):
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(15, 7))
    
    error = np.power(real_theta-estimated_theta,2)
    mean_error = np.mean(error)
    median_error = np.median(error)
    r2 = r2_score(real_theta, estimated_theta)
    
    ax.plot(time_points, error, '--', lw=2, label="Error", color='blue')
    ax.plot(time_points, [mean_error for i in range(len(error))],'-', lw=2, label="Mean error", color='green')
    ax.plot(time_points, [median_error for i in range(len(error))],'-', lw=2, label="Median error", color='red')
    ax.text(0.05, 0.95, f"$R^2$ score = {r2:.4f}", transform=ax.transAxes,
            fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))
    plt.title("Squared error between real and estimated angle (rad)")
    plt.xlabel("Time")
    plt.ylabel("Error")
    plt.legend()
    
    plt.savefig(file_name, dpi=1300)
    plt.close()

In [14]:
# animate_pendulum_single(train_joints_over_time,"single_single_pendulum_train.mp4")
# animate_pendulum_single(test_joints_over_time,"single_single_pendulum_test.mp4")

In [15]:
def iteratively_fit_sindy(true_derivatives, dt, function_library, train_noise, x_train, if_fit_with_derivatives=False, max_t=1, num=50, **kwargs):
    best_t = -1
    best_score = -np.inf
    best_alpha = -1
    best_model = None
    
    train_data = x_train + train_noise if train_noise is not None else x_train

    for t in np.linspace(1e-4, max_t, num):
        for alpha in np.linspace(0.005, 0.9, 30):
            model = ps.SINDy(feature_library=function_library,optimizer=ps.optimizers.STLSQ(threshold=t, max_iter=35, alpha=alpha, verbose=False),  feature_names=['theta_1', 'omega_1'], **kwargs)
            
            if if_fit_with_derivatives:
                model.fit(np.nan_to_num(train_data, posinf=0, neginf=0, nan=0, copy=False).T, t=dt, x_dot=true_derivatives)
            else:
                model.fit(np.nan_to_num(train_data, posinf=0, neginf=0, nan=0, copy=False).T, t=dt)
            
            score = model.score(x_test.T, t=dt, x_dot=true_derivatives)
                        
            if(score > best_score):
                best_score = score
                best_t = t
                best_model = model
                best_alpha = alpha
            
    print(f"Best threshold: {best_t}")
    print(f"Best score: {best_score}")
    print(f"Best alpha: {best_alpha}")
    best_model.print()
    
    return best_model 

# Fitting SINDy

In [16]:
standard_trigon_lib = ps.feature_library.FourierLibrary(n_frequencies=2)
standard_polynom_lib = ps.feature_library.PolynomialLibrary(degree=2,include_interaction=True, include_bias=False)
powers = [i for i in range(1,3)]

def compute_trigon_power(t_arg, t_func, power):
    return np.nan_to_num(np.power(1 / t_func(t_arg), power), nan=0, neginf=0, posinf=0)

def compute_power(arg, power):
    return np.nan_to_num(np.power(1 / arg, power), nan=0, neginf=0, posinf=0)

inverse_trigon = [
    *[lambda x: compute_trigon_power(x, np.sin, power) for power in powers], 
    *[lambda x,y: compute_trigon_power(x*y, np.sin, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x/y, np.sin, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x + y, np.sin, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x - y, np.sin, power) for power in powers],
    
    *[lambda x: compute_trigon_power(x, np.cos, power) for power in powers], 
    *[lambda x,y: compute_trigon_power(x*y, np.cos, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x/y, np.cos, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x + y, np.cos, power) for power in powers],
    *[lambda x,y: compute_trigon_power(x - y, np.cos, power) for power in powers],
]

inverse_trigon_names = [
    *[lambda x: f"(1/sin({x}))^{power}" for power in powers], 
    *[lambda x,y: f"(1/sin({x}*{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/sin({x}/{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/sin({x}+{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/sin({x}-{y}))^{power}" for power in powers],
    
    *[lambda x: f"(1/cos({x}))^{power}" for power in powers], 
    *[lambda x,y: f"(1/cos({x}*{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/cos({x}/{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/cos({x}+{y}))^{power}" for power in powers],
    *[lambda x,y: f"(1/cos({x}-{y}))^{power}" for power in powers],
]

inverse_polynom = [
    *[lambda x: compute_power(x, power) for power in powers], 
    *[lambda x,y: compute_power(x*y, power) for power in powers],
    *[lambda x,y: compute_power(x/y, power) for power in powers],
    *[lambda x,y: compute_power(x+y, power) for power in powers],
    *[lambda x,y: compute_power(x-y, power) for power in powers]
]

inverse_polynom_names = [
    *[lambda x: f"(1/{x})^{power}" for power in powers], 
    *[lambda x,y: f"(1/{x}*{y})^{power}" for power in powers],
    *[lambda x,y: f"(1/{x}/{y})^{power}" for power in powers],
    *[lambda x,y: f"(1/{x}+{y})^{power}" for power in powers],
    *[lambda x,y: f"(1/{x}-{y})^{power}" for power in powers]
]

custom_trigon_lib = ps.feature_library.CustomLibrary(library_functions=inverse_trigon, function_names=inverse_trigon_names, interaction_only=False)
custom_polynom_lib = ps.feature_library.CustomLibrary(library_functions=inverse_polynom, function_names=inverse_polynom_names, interaction_only=False)

# standard_lib =  standard_trigon_lib + standard_polynom_lib
# all_libs =  custom_trigon_lib * standard_lib + custom_polynom_lib * standard_lib + standard_lib + standard_trigon_lib * standard_polynom_lib  
all_libs =  standard_trigon_lib + standard_polynom_lib + standard_trigon_lib * standard_polynom_lib 

## Differentiating on-the-fly

In [17]:
model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=0.17, max_iter=35, alpha=0.4, verbose=True), feature_names=['theta_1', 'omega_1'])
model.fit(np.nan_to_num(x_train, posinf=0, neginf=0, nan=0, copy=False).T, t=dt)
model.print()

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 2.1967e+03 ... 2.6692e-01 ...         10 ... 2.1970e+03
         1 ... 2.7487e+02 ... 7.0275e-01 ...          6 ... 2.7557e+02
         2 ... 5.2071e-03 ... 9.3243e-01 ...          6 ... 9.3764e-01
(theta_1)' = 1.000 omega_1
(omega_1)' = -2.450 sin(1 theta_1)


In [ ]:
blind_simulation, info = model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
blind_simulation = blind_simulation.T
blind_simulation[0] = loop_around_angles(blind_simulation[0])

joints_over_time_blind = compute_joints_position(blind_simulation[0])

In [ ]:
animate_pendulum_versus(test_joints_over_time,joints_over_time_blind,"blind_single_pendulum.mp4", interval=0.01, fps=30)
draw_state_diagrams(x_test_theta, blind_simulation[0], t_test, "blind_single_simulation.png")

## Fitting with known derivatives


In [ ]:
model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=0.2, max_iter=35, alpha=0.08, verbose=True),  feature_names=['theta_1', 'omega_1'])
model.fit(np.nan_to_num(x_train, posinf=0, neginf=0, nan=0, copy=False).T, t=dt,x_dot=true_derivatives, unbias=True)
model.print()

In [ ]:
model = iteratively_fit_sindy(true_derivatives,dt, all_libs, None,x_train, True, max_t=0.2,num=30)

In [ ]:
informed_simulation, info = model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
informed_simulation = informed_simulation.T
informed_simulation[0] = loop_around_angles(informed_simulation[0])

joints_over_time_informed = compute_joints_position(informed_simulation[0])

In [ ]:
animate_pendulum_versus(test_joints_over_time, joints_over_time_informed,"informed_single_pendulum.mp4", interval=0.003, fps=30)
draw_state_diagrams(x_test_theta, informed_simulation[0], t_test, "informed_single_simulation.png")

## Fitting with noise

In [98]:
noise_magnitude = 0.01
train_noise = np.random.normal(0, 1, x_train.shape) * noise_magnitude

### Just noisy

In [105]:
model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=0.2, max_iter=35, alpha=0.015, verbose=True),  feature_names=['theta_1','omega_1'])
model.fit(np.nan_to_num((x_train + train_noise), posinf=0, neginf=0, nan=0, copy=False).T, t=dt)
model.print()

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 6.0491e+05 ... 5.5649e+01 ...         99 ... 6.0497e+05
         1 ... 6.0471e+05 ... 5.5712e+01 ...         98 ... 6.0477e+05
         2 ... 6.0445e+05 ... 5.5714e+01 ...         98 ... 6.0451e+05
(theta_1)' = 49016.686 sin(1 theta_1) + -40107715.625 cos(1 theta_1) + 2351369.288 sin(1 omega_1) + 49699578.994 cos(1 omega_1) + -76433.949 sin(2 theta_1) + -6646432.639 cos(2 theta_1) + -411185.117 sin(2 omega_1) + -2953103.485 cos(2 omega_1) + -922064.921 omega_1 + -7071085.997 theta_1^2 + -8339.226 theta_1 omega_1 + 6166771.915 omega_1^2 + -22373140.268 sin(1 theta_1) theta_1 + 11920.563 sin(1 theta_1) omega_1 + 22379.960 sin(1 theta_1) theta_1 omega_1 + 1255.162 sin(1 theta_1) omega_1^2 + 59708.027 cos(1 theta_1) theta_1 + 12269.810 cos(1 theta_1) omega_1 + 1405184.460 cos(1 theta_1) theta_1^2 + -5426.872 cos(1 theta_1) theta_1 omega_1 + -56997.005 cos(1 theta_1) omega_1^2 + 11

In [106]:
best_model = iteratively_fit_sindy(true_derivatives, dt, all_libs, train_noise, x_train, False, 0.5,60)

Best threshold: 0.5
Best score: -8.374394777531787
Best alpha: 0.9
(theta_1)' = -9.445 cos(2 theta_1) + 1.883 omega_1 + -48.813 omega_1^2 + 4.662 sin(1 theta_1) theta_1 omega_1 + -3.527 cos(1 omega_1) omega_1 + 70.605 cos(1 omega_1) theta_1^2 + -124.070 cos(1 omega_1) omega_1^2 + -17.585 sin(2 theta_1) theta_1 + 41.971 cos(2 theta_1) theta_1^2 + 81.938 sin(2 omega_1) omega_1 + -1.966 sin(2 omega_1) theta_1^2 + 3.732 cos(2 omega_1) omega_1 + -31.428 cos(2 omega_1) theta_1^2 + -46.628 cos(2 omega_1) omega_1^2
(omega_1)' = 6.845 theta_1 omega_1 + -10.087 cos(1 theta_1) theta_1 omega_1 + -10.864 cos(1 omega_1) theta_1 omega_1 + -2.692 sin(2 theta_1) theta_1^2 + 3.023 sin(2 omega_1) theta_1


In [112]:
noisy_simulation, info = best_model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
noisy_simulation = noisy_simulation.T
noisy_simulation[0] = loop_around_angles(noisy_simulation[0])

joints_over_time_noisy = compute_joints_position(noisy_simulation[0])

In [ ]:
animate_pendulum_versus(test_joints_over_time,joints_over_time_noisy,"noisy_pendulum.mp4", interval=0.001, fps=40)

In [115]:
draw_state_diagrams(x_test_theta, noisy_simulation[0], t_test, "noisy_simulation.png")
plot_progressive_erros(x_test_theta, noisy_simulation[0], t_test, "noisy_errors.png")

### Applying smoother

In [79]:
model = ps.SINDy(feature_library=all_libs,optimizer=ps.optimizers.STLSQ(threshold=0.45, max_iter=35, alpha=0.025, verbose=True),  feature_names=['theta_1','omega_1'],differentiation_method=ps.differentiation.SmoothedFiniteDifference(smoother_kws={'window_length': 6, 'polyorder':1,'mode':'nearest'}))
model.fit(np.nan_to_num((x_train + train_noise), posinf=0, neginf=0, nan=0, copy=False).T, t=dt)
model.print()

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 1.5908e+03 ... 6.7482e-02 ...          7 ... 1.5909e+03
         1 ... 1.3061e+03 ... 6.4321e-02 ...          3 ... 1.3062e+03
         2 ... 1.2229e+03 ... 7.9556e-02 ...          2 ... 1.2229e+03
         3 ... 1.2191e+03 ... 1.4310e-01 ...          1 ... 1.2192e+03
         4 ... 1.2183e+03 ... 1.4873e-01 ...          1 ... 1.2184e+03
(theta_1)' = 0.000
(omega_1)' = -2.439 sin(1 theta_1)


/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.45) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.45) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.45) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.45) and eliminated all coefficients
  warnings.warn(


In [121]:
best_model = iteratively_fit_sindy(true_derivatives, dt, all_libs, train_noise, x_train, False, 0.55, 60, differentiation_method=ps.differentiation.SmoothedFiniteDifference(smoother_kws={'window_length': 7, 'polyorder':1,'mode':'nearest'}))

/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.5406796610169492) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.55) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.55) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.55) and eliminated all coefficients
  w

Best threshold: 0.36359322033898306
Best score: 0.9267900656490877
Best alpha: 0.8382758620689655
(theta_1)' = 0.461 sin(2 omega_1) theta_1^2 + -1.195 cos(2 omega_1) omega_1
(omega_1)' = -2.446 sin(1 theta_1)


/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.55) and eliminated all coefficients
  warnings.warn(
/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem2/MMML/project/poc/.venv/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.55) and eliminated all coefficients
  warnings.warn(


In [126]:
smoothed_noisy_simulation, info = best_model.simulate(test_initial_pendulum_config, t_test, integrator="odeint", integrator_kws=odeint_integrator_args)
smoothed_noisy_simulation = smoothed_noisy_simulation.T
smoothed_noisy_simulation[0] = loop_around_angles(smoothed_noisy_simulation[0])

joints_over_time_noisy_smmothed = compute_joints_position(smoothed_noisy_simulation[0])

In [129]:
animate_pendulum_versus(test_joints_over_time,joints_over_time_noisy_smmothed, "smoothed_noisy_pendulum.mp4", interval=0.003, fps=30)

In [128]:
draw_state_diagrams(x_test_theta, smoothed_noisy_simulation[0], t_test, "smoothed_noisy_simulation.png")
plot_progressive_erros(x_test_theta, smoothed_noisy_simulation[0], t_test, "smoothed_noisy_errors.png")